In [ ]:
import numpy as np
from Source import RS64_Encoder
from Source import RS64_Decoder
from Source.GF64 import GFE

import keras
from keras.layers import Dense, Input,Conv1D, MaxPooling1D, Flatten, Embedding, Reshape, ReLU, Add, Concatenate, Average
from keras.models import Sequential, Model
from keras.models import load_model

In [ ]:
m = 6
t_RS = 8
b_RS = 1
order_alpha=(2**m)-1
length_parity=2*t_RS
n_RS=order_alpha
k_RS=n_RS - length_parity

Z_RS=[]
for i in range(2*t_RS):    
    Z_RS.append((b_RS+i)%order_alpha)
    
g_x = RS64_Encoder.Generation_Polynomial(Z_RS, order_alpha)

In [ ]:
codeword_size = 10**7

In [ ]:
model = load_model('model/model.h5')

In [ ]:
for i in range(20):
    cx = np.random.randint(2, size=(codeword_size, 378))
    cx = np.where(cx > 0, -1, 1)
    cx = cx.reshape((codeword_size, -1))
    
    Eb_No = 3
    rx = RS64_Decoder.Receive_Bit(Eb_No, cx, k_RS, n_RS)
    
    test_cx = np.random.randint(2, size=(10**2, 378))
    test_cx = np.where(test_cx > 0, -1, 1)
    test_cx = cx.reshape((codeword_size, -1))
    
    test_rx = RS64_Decoder.Receive_Bit(Eb_No, test_cx, k_RS, n_RS)
    
    model.fit(rx, cx, 
          epochs=100,
          batch_size=10000,
          shuffle=True,
          validation_data=(test_rx, test_cx), verbose = 0)
    
    if run%1 == 0:
        loss = model.evaluate(x_test_noisy, x_test)
        print(loss)
        model.save('model/model.h5')
        print('save_model')

In [ ]:
loss = model.evaluate(x_test_noisy, x_test)
print(loss)
model.save('model/model.h5')
print('save_model')